In [43]:
!pip install tez

In [44]:
import os
import albumentations
import matplotlib.pyplot as plt
import pandas as pd

import tez
from tez.datasets import ImageDataset
from tez.callbacks import EarlyStopping

import torch
import torch.nn as nn

import torchvision

from sklearn import metrics, model_selection

%matplotlib inline

In [45]:
df = pd.read_csv('../input/academia-connect-nvidia/input/train.csv')
df

,Image_path,label
0,/Users/sunita/Documents/Nvidia-Hackathon-Datas...,0
1,/Users/sunita/Documents/Nvidia-Hackathon-Datas...,0
2,/Users/sunita/Documents/Nvidia-Hackathon-Datas...,0
3,/Users/sunita/Documents/Nvidia-Hackathon-Datas...,0
4,/Users/sunita/Documents/Nvidia-Hackathon-Datas...,0
...,...,...
5968,/Users/sunita/Documents/Nvidia-Hackathon-Datas...,-1
5969,/Users/sunita/Documents/Nvidia-Hackathon-Datas...,-1
5970,/Users/sunita/Documents/Nvidia-Hackathon-Datas...,-1
5971,/Users/sunita/Documents/Nvidia-Hackathon-Datas...,-1


In [46]:
train = df[df['label'] != -1]
train

,Image_path,label
0,/Users/sunita/Documents/Nvidia-Hackathon-Datas...,0
1,/Users/sunita/Documents/Nvidia-Hackathon-Datas...,0
2,/Users/sunita/Documents/Nvidia-Hackathon-Datas...,0
3,/Users/sunita/Documents/Nvidia-Hackathon-Datas...,0
4,/Users/sunita/Documents/Nvidia-Hackathon-Datas...,0
...,...,...
5870,/Users/sunita/Documents/Nvidia-Hackathon-Datas...,1
5871,/Users/sunita/Documents/Nvidia-Hackathon-Datas...,1
5872,/Users/sunita/Documents/Nvidia-Hackathon-Datas...,1
5873,/Users/sunita/Documents/Nvidia-Hackathon-Datas...,1


In [47]:
df_train, df_valid = model_selection.train_test_split(
    train,
    test_size = 0.1,
    random_state = 42,
    stratify = train.label.values
)

In [48]:
image_path = '../input/academia-connect-nvidia/'
train_image_paths = [
    os.path.join(image_path, '/'.join(x.split('/')[-3:])) for x in df_train.Image_path.values
]
valid_image_paths = [
    os.path.join(image_path, '/'.join(x.split('/')[-3:])) for x in df_valid.Image_path.values
]

# targets for training
train_targets = df_train.label.values

# targets for validation
valid_targets = df_valid.label.values

In [49]:
import cv2
def get_img_shape(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img.shape

In [50]:
# Let's create training and validation datasets
# Tez provides simple dataset class that you can use directly

# we create the train_dataset
train_dataset = ImageDataset(
    image_paths=train_image_paths,
    targets=train_targets,
    augmentations=None,
)

# and the validation dataset
valid_dataset = ImageDataset(
    image_paths=valid_image_paths,
    targets=valid_targets,
    augmentations=None,
)

# note that we have resized the images to 256x256
# and we are not using any augmentations
# we will come back to that later

In [51]:
# thus, we have image and targets
# super-easy!

# Let's see some images!

def plot_image(img_dict):
    image_tensor = img_dict["image"]
    target = img_dict["targets"]
    print(target)
    plt.figure(figsize=(10, 10))
    image = image_tensor.permute(1, 2, 0) / 255
    plt.imshow(image)

In [52]:
#Let's define a model now
# We inherit from tez.Model instead of nn.Module
# we have monitor_metrics if we want to monitor any metrics
# except the loss
# and we return 3 values in forward function.

class LeafModel(tez.Model):
    def __init__(self, num_classes):
        super().__init__()

        self.convnet = torchvision.models.resnet18(pretrained=True)
        self.convnet.fc = nn.Linear(512, num_classes)
        self.step_scheduler_after = "epoch"
        
    def monitor_metrics(self, outputs, targets):
        if targets is None:
            return {}
        outputs = torch.argmax(outputs, dim=1).cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, outputs)
        return {"accuracy": accuracy}
    
    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=1e-3)
        return opt
    
    def fetch_scheduler(self):
        sch = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=0.7)
        return sch

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape

        outputs = self.convnet(image)
        
        if targets is not None:
            loss = nn.CrossEntropyLoss()(outputs, targets)
            metrics = self.monitor_metrics(outputs, targets)
            return outputs, loss, metrics
        return outputs, None, None

In [53]:
model = LeafModel(num_classes=5)

In [54]:

image = train_dataset[0]["image"].unsqueeze(0)
target = train_dataset[0]["targets"].unsqueeze(0)


model(image, target)

(tensor([[-0.3128, -0.6381,  0.5704,  0.0675,  0.7750]],
        grad_fn=<AddmmBackward>),
 tensor(2.1485, grad_fn=<NllLossBackward>),
 {'accuracy': 0.0})

In [55]:
train_aug = albumentations.Compose([
            albumentations.RandomResizedCrop(64, 64),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            )], p=1.)
      
valid_aug = albumentations.Compose([
            albumentations.CenterCrop(64, 64, p=1.),
            albumentations.Resize(64, 64),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            )], p=1.)

train_dataset = ImageDataset(
    image_paths=train_image_paths,
    targets=train_targets,
    augmentations=train_aug,
)

valid_dataset = ImageDataset(
    image_paths=valid_image_paths,
    targets=valid_targets,
    augmentations=valid_aug,
)
es = EarlyStopping(
    monitor="valid_accuracy", model_path="model.bin", patience=2, mode="max"
)
model.fit(
    train_dataset,
    valid_dataset=valid_dataset,
    train_bs=32,
    valid_bs=64,
#     device="cuda",
    device="cpu",
    epochs=5,
    callbacks=[es],
    fp16=True,
)

/opt/conda/lib/python3.7/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
  0%|          | 0/166 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/torch/cuda/amp/autocast_mode.py:114: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")
100%|██████████| 10/10 [00:03<00:00,  3.05it/s, accuracy=0.938, loss=0.209, stage=valid]


Validation score improved (-inf --> 0.9375). Saving model!


100%|██████████| 10/10 [00:02<00:00,  3.42it/s, accuracy=0.9, loss=0.254, stage=valid] 


EarlyStopping counter: 1 out of 2


100%|██████████| 10/10 [00:02<00:00,  3.36it/s, accuracy=0.955, loss=0.141, stage=valid]


Validation score improved (0.9375 --> 0.9546875). Saving model!


100%|██████████| 10/10 [00:02<00:00,  3.69it/s, accuracy=0.959, loss=0.14, stage=valid]


Validation score improved (0.9546875 --> 0.959375). Saving model!


100%|██████████| 10/10 [00:02<00:00,  3.63it/s, accuracy=0.938, loss=0.171, stage=valid]

EarlyStopping counter: 1 out of 2


In [74]:
test = df[df['label'] == -1]

image_path = '../input/academia-connect-nvidia/'

test_image_paths = [
    os.path.join(image_path, '/'.join(x.split('/')[-2:])) for x in test.Image_path.values
]

In [78]:
import numpy as np
test_targets = test.label.values
test_targets = np.zeros_like(test_targets)

test_aug = albumentations.Compose([
            albumentations.CenterCrop(64, 64, p=1.),
            albumentations.Resize(64, 64),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            )], p=1.)

test_dataset = ImageDataset(
    image_paths=test_image_paths,
    targets=test_targets,
    augmentations=test_aug,
)

In [79]:
preds = model.predict(test_dataset, batch_size=32, n_jobs=-1)
final_preds = None
for p in preds:
    if final_preds is None:
        final_preds = p
    else:
        final_preds = np.vstack((final_preds, p))
final_preds = final_preds.argmax(axis=1)
test.label = final_preds

100%|██████████| 4/4 [00:00<00:00,  5.58it/s, stage=test]
/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [80]:
test.to_csv("submission.csv", index=False)

In [81]:
submission = pd.read_csv('./submission.csv')

In [83]:
submission.label.value_counts()

0    60
1    38
Name: label, dtype: int64

In [85]:
submission['Image_name'] = submission['Image_path'].apply(lambda x: x.split('/')[-1])

In [87]:
submission = submission[['Image_name', 'label']]

In [93]:
submission

,Image_name,label
0,63.jpg,1
1,77.jpg,0
2,88.jpg,0
3,89.jpg,0
4,76.jpg,0
...,...,...
93,90.jpg,1
94,91.jpg,0
95,85.jpg,0
96,52.jpg,1


In [104]:
json_dict = {}
for name, label in zip(submission.Image_name.values, submission.label.values):
    json_dict[name] = int(label)

In [105]:
import json
with open('./submission.json', 'w') as json_file:
    json.dump(json_dict, json_file)